In [1]:
# 저번 튜토리얼을 통해 텐서플로우를 쓰는것이 코드를 간소화 할 수 있고 
# 그렇기 때문에 세세한 구현보다 모델의 구조에 더 집중할 수 있다는 점을 배웠습니다
# 이번에는 기본적인 Convolutional Neural Network를 짜보면서 
# tf.nn.conv2d(), tf.nn.max_pool() 같은 함수들을 배워보겠습니다

In [2]:
# 이번 튜토리얼에서 사용할 데이터는 mnist 데이터로 0~9사이의 28x28 크기의 손글씨 데이터입니다
# 하도 유명한 데이터이기 때문에 텐서플로우에서 따로 데이터를 받고 불러오는 함수를 제공합니다
# one_hot=True는 만약 그 데이터가 3으로 라벨링 되어있으면 [0,0,0,1,0,0,0,0,0,0]로 바꿔줍니다

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# 데이터를 불러왔으니 이제 모델을 만들어보겠습니다
# 저번에는 모델을 만들고 세션을 열어서 학습시켰으나
# 이번에는 tf.InteractiveSession()을 사용해 세션 안에서 모델 또한 만들어보겠습니다
# InteractiveSession은 현재 열린 세션을 default 세션으로 인식하기 때문에
# sess.run()을 통해 결과값을 얻지 않아도 되고 Tensor.eval()로 바로 값을 구하거나
# Operation.run()을 통해 함수를 돌릴 수 있습니다

import tensorflow as tf
sess = tf.InteractiveSession()

In [4]:
# 먼저 인풋과 라벨을 받을 tf.placeholder를 만듭니다 shape에서 None은 값을 동적으로 할당한다는
# 뜻으로 여기서는 한번에 들어오는 batch의 크기에 따라 동적으로 모양을 정할 수 있게 해줍니다
# tf.reshape(tensor, shape, name=None)는 텐서의 모양을 바꾸고 싶을때 사용하는 함수입니다
# 여기서 -1은 shape에서 나머지 숫자들이 다 정해져 있으면 input의 크기에 맞춰 -1 부분에 넣는다는 뜻입니다 
# 예를 들어 x가 [5,784]로 들어왔다면 reshape는 이를 [5,28,28,1]로 바꿔줍니다

x  = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1,28,28,1])

In [5]:
# 이번에는 자주쓰는 함수들을 좀 더 간편하게 정의해보도록 하겠습니다 예를들어 weight를 초기화하려면
# w1=tf.Variable(tf.truncated_normal(shape,stddev=0.1))처럼 복잡하게 되는데
# 매번 이렇게 치려면 복잡하고 길어지기 때문에 줄여보겠습니다
# 참고로 tf.truncated_normal(shape, mean=0.0, stddev=1.0, ..., name=None)은
# 평균 얼마에 표준편차 얼마를 가지는 값들을 shape에 맞게 리턴해줍니다
# bias도 위와 같이 간단하게 만들어줍니다

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [6]:
# 이제 convolution 함수와 pooling 함수를 만들어 보겠습니다
# 텐서플로우는 자체적으로 이 함수들을 제공하는데 들어가는 인자가 몇가지 있습니다
# tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
# input -> [batch, in_height, in_width, in_channels]
# filter -> [filter_height, filter_width, in_channels, out_channels]
# strides -> [batch, in_height, in_width, in_channels]이고 [1,x,x,1]의 형태를 가집니다
# padding -> "same" or "valid"로 same이면 인풋크기가 줄어들지 않습니다

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [7]:
# tf.nn.max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)
# ksize는 한번 pool하는 범위, strides와 padding은 conv2d와 같습니다

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [8]:
# 자 이제 준비가 되었으니 모델을 만들어보겠습니다
# 형태는 간단히 (conv -> relu -> pool)*2 -> fc1 -> pool -> drop -> fc2 로 갑니다
# 한단계 한단계씩 해보겠습니다. 첫번째 conv->relu->pool을 거치면 
# 모양이 [-1,28,28,1]에서 [-1,14,14,32]가 됩니다
 
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [9]:
# 두번째 conv->relu->pool을 거치면 
# 모양이 [-1,14,14,32]에서 [-1,7,7,64]가 됩니다

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [10]:
# 이제 fc(fully connected layer)로 넘어갑니다
# fc는 1차원 벡터를 사용하므로 conv를 통해 넘어온 input을 한줄로 reshape 해줍니다
# 그러면 batch dimension은 유지되기 때문에 input은 [-1,7,7,64]에서 [-1,7*7*64]가 됩니다 
# 그리고 fc를 거치면 [-1,7*7*64]에서 [-1,1024]가 됩니다

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [11]:
# 여기서 한번 dropout을 해줍니다 
# dropout은 regularization을 통해 overfitting을 막아주는 효과가 있습니다
# 텐서플로우에서 dropout은 다음과 같이 구현되어 있습니다
# tf.nn.dropout(x, keep_prob, noise_shape=None, seed=None, name=None)
# keep_prob은 확률이므로 0~1의 값을 가집니다

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [12]:
# 마지막으로 fc를 거칩니다

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [13]:
# 이제 loss를 계산하고 최적화를 시킬 차례입니다
# softmax를 통해 cross entropy loss를 계산하고 이를 AdamOptimizer로 최적화힙니다

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [14]:
# 그리고 정확도를 계산하기 위해 예상값과 실제 라벨을 비교해 True/ False로 판별된 정보를
# 0과 1로 바꿔 정확도를 계산합니다
# tf.equal(x, y, name=None)
# tf.argmax(input, axis=None, name=None, dimension=None) -> 가장 큰 값의 index

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
# variable을 초기화 시켜줍니다
# interactive session이기 때문에 operation.run() 하면 됩니다

tf.global_variables_initializer().run()

In [16]:
# 이제 실제 학습을 시켜보겠습니다
# 한번에 다 돌릴 수 없기 때문에 batch를 사용하는데 mnist에는 친절하게
# mnist.train.next_batch(#)가 구현되어 있습니다 하지만 다른 데이터를 쓸때는
# 전체 데이터를 분할해서 차례대로 feed_dict를 통해 넘겨줘야 합니다
# feed_dict={x: , y: , keep_prob: }를 인자로 전달합니다
# 100번마다 정확도도 출력해줍니다

for i in range(20000):
	batch = mnist.train.next_batch(50)
	if i%100 == 0:
		train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
		print("step %d, training accuracy %g"%(i, train_accuracy))
	train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.06
step 100, training accuracy 0.84
step 200, training accuracy 0.96
step 300, training accuracy 0.86
step 400, training accuracy 0.94
step 500, training accuracy 0.9
step 600, training accuracy 1
step 700, training accuracy 0.94
step 800, training accuracy 0.92
step 900, training accuracy 1
step 1000, training accuracy 0.96
step 1100, training accuracy 0.94
step 1200, training accuracy 0.94
step 1300, training accuracy 0.98
step 1400, training accuracy 1
step 1500, training accuracy 1
step 1600, training accuracy 0.96
step 1700, training accuracy 1
step 1800, training accuracy 0.94
step 1900, training accuracy 1
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 0.96
step 2300, training accuracy 1
step 2400, training accuracy 0.96
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000, training accuracy 

In [18]:
# 마지막으로 test data를 불러와서 얼마나 학습이 되었는지 확인해봅니다
# 그냥 돌리면 제 노트북의 메모리가 터지기 때문에 분할해서 평균을 내보도록 하겠습니다

batch_size=200
acc_total=0

j=0
while j<len(mnist.test.images):
    test_input = mnist.test.images[j:j+batch_size]
    test_output= mnist.test.labels[j:j+batch_size]
    batch_acc=accuracy.eval(feed_dict={x: test_input, y_: test_output, keep_prob: 1.0})
    acc_total+=batch_acc
    j+=batch_size  
    
num_ops=len(mnist.test.images)/batch_size
print("test accuracy: %f"% (acc_total/num_ops))

test accuracy: 0.991100
